## README 
Website: https://www.eqtlgen.org/phase1.html

Paper: https://www.nature.com/articles/s41588-021-00913-zhttps://www.nature.com/articles/s41588-021-00913-z

This Jupyter notebook README covers cis-eQTL and trans-eQTL results from the eQTLGen project. The dataset includes significant files for both cis-eQTL and trans-eQTL analyses. 

Files contain various columns:


1. Pvalue - P-value
2. SNP - SNP rs ID
3. SNPChr - SNP chromosome
4. SNPPos - SNP position
5. AssessedAllele - Assessed allele, the Z-score refers to this allele
6. OtherAllele - Not assessed allele
7. Zscore - Z-score
8. Gene - ENSG name (Ensembl v71) of the eQTL gene
9. GeneSymbol - HGNC name of the gene
10. GeneChr - Gene chromosome
11. GenePos - Centre of gene position
12. NrCohorts - Total number of cohorts where this SNP-gene combination was tested
13. NrSamples - Total number of samples where this SNP-gene combination was tested
14. FDR - False discovery rate estimated based on permutations
15. BonferroniP - P-value after Bonferroni correction

The cis-eQTL analysis includes 19,250 genes expressed in blood, with SNP-gene combinations within 1Mb from the gene center and tested in at least 2 cohorts. The trans-eQTL analysis tests 19,960 genes expressed in blood and 10,317 trait-associated SNPs based on GWAS Catalog, Immunobase, and Astle et al. study. Trans-eQTL combinations have a distance of >5Mb and were tested in at least 2 cohorts.

The FDR calculation uses a pruned set of SNPs for trans-eQTL mapping and permutation-based FDR calculation. Crossmapping filters are applied to identify and remove potential artifacts in trans-eQTL results, recalculating the FDR afterward. Note that the full results file has not been filtered for cross-mapping effects, which may lead to artifacts in the data.

The code below demonstrates the process of creating a graph-based representation of the combined cis and trans-eQTL data using PyTorch Geometric. This process can be broken down into several steps:

1. Combine cis and trans dataframes: The code begins by concatenating the cis and trans dataframes into a single dataframe named 'data', which contains information on both cis-eQTL and trans-eQTL results. This combined dataset simplifies the process of working with the data and ensures that all relevant information is contained within a single data structure.

2. Create mappings for genes and SNPs: To represent the genes and SNPs as nodes in the graph, integer indices are assigned to each unique gene and SNP. This is done using dictionaries called 'gene_to_idx' and 'snp_to_idx', which map the gene and SNP identifiers to their corresponding integer indices.

3. Generate node type labels: Node type labels are created using PyTorch tensors, distinguishing between gene nodes (assigned a label of 0) and SNP nodes (assigned a label of 1). This differentiation is useful for various graph-based analyses and machine learning tasks that require knowledge of node types.

4. Create edges based on gene and SNP indices: Edges in the graph represent the relationships between genes and SNPs. These edges are created by iterating over the 'data' dataframe and extracting the corresponding gene and SNP indices from the previously created mappings. The edges are then represented as a PyTorch tensor with a long data type.

5. Convert edges to undirected: Since the relationships between genes and SNPs are undirected, the edges in the graph should also be undirected. This is achieved using the 'to_undirected()' function from PyTorch Geometric, which ensures that the graph correctly represents the underlying biology.

6. Create a PyTorch Geometric graph: Finally, the graph is created using the PyTorch Geometric 'Data' class. The node types and edge indices are used as inputs to instantiate the graph object, which can then be utilized for further analysis and visualization.

The resulting 'graph' object is a PyTorch Geometric representation of the combined cis and trans-eQTL data. The prediction task is to predict new association edges given the training edges, with the task type being link prediction. Below are a few important graph statistics:

- Number of nodes: 3681495
- Number of SNP nodes: 3664025
- Number of Gene nodes: 17470
- Number of edges: 10567450
- Number of connected components: 424
- Average degree: 5.74
- Median degree: 2.0
- Standard deviation of degree: 69.81
- Density: 0.0000015594
- Assortativity: -0.2267915607

The prediction task is to predict new association edges given the training edges, with the task type being link prediction. The data splitting is random while maintaining an equal proportion of cis- and trans- associations.

## Data Setup 

### Libraries

In [1]:
import os

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, average_precision_score

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.utils import to_undirected, negative_sampling

import networkx as nx
from ogb.io import DatasetSaver
from ogb.linkproppred import LinkPropPredDataset

In [2]:
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Geometric version: {torch_geometric.__version__}")

PyTorch version: 2.0.0+cu118
PyTorch Geometric version: 2.3.1


### Load data

In [6]:
# Read files
pf = pq.ParquetFile("big-cis.parquet")
table = pf.read(columns=['SNP', 'Gene', 'FDR', 'BonferroniP'])
cis = table.to_pandas()

pf = pq.ParquetFile("big-trans.parquet")
table = pf.read(columns=['SNP', 'Gene', 'FDR', 'BonferroniP'])
trans = table.to_pandas()

pf = pq.ParquetFile("sig-cis.parquet")
table = pf.read(columns=['SNP', 'Gene', 'FDR', 'BonferroniP'])
sig_cis = table.to_pandas()

pf = pq.ParquetFile("sig-trans.parquet")
table = pf.read(columns=['SNP', 'Gene', 'FDR', 'BonferroniP'])
sig_trans = table.to_pandas()

In [7]:
def filter_fdr(df):
    threshold = 0.05
    return df[df['FDR'] < threshold]

# Load cis and trans dataframes
cis_unfiltered = sig_cis
trans_unfiltered = sig_trans

# Filter cis and trans dataframes based on FDR
cis_filtered = filter_fdr(cis_unfiltered)
trans_filtered = filter_fdr(trans_unfiltered)

# Define cis and trans as the difference between unfiltered and filtered dataframes
cis_unfiltered = cis[~cis.index.isin(cis_filtered.index)]
trans_unfiltered = trans[~trans.index.isin(trans_filtered.index)]

# Print lengths of filtered and unfiltered cis and trans dataframes
print(f"Cis filtered: {len(cis_filtered)}")
print(f"Trans filtered: {len(trans_filtered)}")
print(f"Cis unfiltered: {len(cis_unfiltered)}")
print(f"Trans unfiltered: {len(trans_unfiltered)}")
print(f"Cis: {len(cis)}")
print(f"Trans: {len(trans)}")

Cis filtered: 10507664
Trans filtered: 59786
Cis unfiltered: 116834134
Trans unfiltered: 203487960
Cis: 127341798
Trans: 203547746


### Create graph

In [ ]:
# Combine the cis and trans dataframes
data = pd.concat([cis, trans], ignore_index=True)

# Create mappings for genes and SNPs to integer indices
genes = data['Gene'].unique()
snps = data['SNP'].unique()
gene_to_idx = {gene: idx for idx, gene in enumerate(genes)}
snp_to_idx = {snp: idx + len(genes) for idx, snp in enumerate(snps)}

# Create node type labels
node_types = torch.tensor([0] * len(genes) + [1] * len(snps), dtype=torch.long)

# Create edges
edges = data.apply(lambda row: (gene_to_idx[row['Gene']], snp_to_idx[row['SNP']]), axis=1)
edges = edges[cis_filtered.index.union(trans_filtered.index)]  # Only keep edges with FDR < 0.05
edges = torch.tensor(list(edges), dtype=torch.long).t().contiguous()

# Convert edges to undirected
edges = to_undirected(edges)

# Create the PyTorch Geometric graph
graph = Data(x=node_types.view(-1, 1), edge_index=edges)

# Print the number of nodes and edges in the graph
print("Graph created with", len(genes) + len(snps), "nodes and", edges.size(1), "edges.")

def compute_descriptive_stats(cis_filtered, cis_remaining, trans_filtered, trans_remaining):
    # Compute mean, median, and standard deviation for cis_filtered and cis_remaining
    mean_cis_filtered = cis_filtered['FDR'].mean()
    median_cis_filtered = cis_filtered['FDR'].median()
    std_cis_filtered = cis_filtered['FDR'].std()
    
    mean_cis_remaining = cis_remaining['FDR'].mean()
    median_cis_remaining = cis_remaining['FDR'].median()
    std_cis_remaining = cis_remaining['FDR'].std()
    
    # Compute mean, median, and standard deviation for trans_filtered and trans_remaining
    mean_trans_filtered = trans_filtered['FDR'].mean()
    median_trans_filtered = trans_filtered['FDR'].median()
    std_trans_filtered = trans_filtered['FDR'].std()
    
    mean_trans_remaining = trans_remaining['FDR'].mean()
    median_trans_remaining = trans_remaining['FDR'].median()
    std_trans_remaining = trans_remaining['FDR'].std()
    
    # Print the results
    print("Cis Filtered: mean=%.2e, median=%.2e, std=%.2e" % (mean_cis_filtered, median_cis_filtered, std_cis_filtered))
    print("Cis Remaining: mean=%.2e, median=%.2e, std=%.2e" % (mean_cis_remaining, median_cis_remaining, std_cis_remaining))
    print("Trans Filtered: mean=%.2e, median=%.2e, std=%.2e" % (mean_trans_filtered, median_trans_filtered, std_trans_filtered))
    print("Trans Remaining: mean=%.2e, median=%.2e, std=%.2e" % (mean_trans_remaining, median_trans_remaining, std_trans_remaining))


compute_descriptive_stats(cis_filtered, cis_remaining, trans_filtered, trans_remaining)

### Graph stats

In [ ]:
def print_graph_stats(graph, genes, snps):
    G = nx.Graph()
    for edge in graph.edge_index.t().numpy():
        G.add_edge(edge[0], edge[1])

    num_nodes = G.number_of_nodes()
    num_genes = len(genes)
    num_snps = len(snps)
    num_edges = G.number_of_edges()
    num_connected_components = nx.number_connected_components(G)
    average_degree = np.mean([degree for _, degree in G.degree()])
    median_degree = np.median([degree for _, degree in G.degree()])
    std_degree = np.std([degree for _, degree in G.degree()])
    density = nx.density(G)
    assortativity = nx.degree_assortativity_coefficient(G)

    print(f"Number of nodes: {num_nodes}")
    print("Number of SNP nodes:", num_snps)
    print("Number of Gene nodes:", num_genes)
    print(f"Number of edges: {num_edges}")
    print(f"Number of connected components: {num_connected_components}")
    print(f"Average degree: {average_degree:.2f}")
    print(f"Median degree: {median_degree}")
    print(f"Standard deviation of degree: {std_degree:.2f}")
    print(f"Density: {density:.10f}")
    print(f"Assortativity: {assortativity:.10f}")


# For the combined graph
print("Combined graph stats:")
print_graph_stats(graph, genes, snps)